In [1]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer
from sklearn.multioutput import MultiOutputClassifier
import numpy as np


/home/andrewkim/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
distance_mask_token = 0
no_mask_token = 5
max_len = 371
bert_max_len = 373

data2 = np.load('/home/andrewkim/Desktop/GPCRBert/data/final_mask5_class.npy', allow_pickle=True)

data = []
labels = []
for i in range(data2.shape[0]):
    seq = data2[i][1]
    seq_join = ''.join(seq)

    motif = data2[i][2] # req_pre
    motif_join = ''.join(motif) # requ_pre_str
    start_idx = seq_join.find(motif_join)

    seq_list = list(seq_join)
    seq_list[start_idx + distance_mask_token : start_idx + distance_mask_token + no_mask_token] = 'J' * no_mask_token
    seq_list_spaced = ' '.join(seq_list)
    label = motif_join[distance_mask_token: distance_mask_token + no_mask_token]

    tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert')
    seq_tokenized = tokenizer(seq_list_spaced, return_tensors='pt', padding='max_length', max_length=bert_max_len)
    label_spaced = ' '.join(motif_join)
    label_tokenized = tokenizer(label_spaced, return_tensors='pt')

    data.append(seq_tokenized['input_ids'][0])
    labels.append(label_tokenized['input_ids'][0][1+distance_mask_token:1+distance_mask_token+no_mask_token]) #


print(len(data))
print(len(labels))
print(data[0])
print(labels[0])

252
252
tensor([ 2,  6,  6, 14,  9,  8, 24,  8,  8,  7, 21,  7, 11,  8, 21, 10,  5, 11,
         8,  5,  6, 11,  8, 19,  7, 17,  8,  5,  8, 11, 15,  6, 11,  6, 12, 19,
         9, 13,  5, 18, 15,  8, 15, 17, 20, 19, 11, 15, 10,  5,  6, 23,  6, 14,
         5,  8, 21,  7,  5,  6,  8,  8, 16, 19,  7,  6,  6, 23, 11,  5, 15, 12,
        15, 24, 15, 19,  7, 17, 19, 24, 23,  9, 19, 24, 15, 10, 11, 14,  8,  5,
        23,  8, 15,  6, 10, 11,  9, 15,  5, 23,  8,  1,  1,  1,  1,  1, 20, 19,
         6, 11, 15, 10, 16, 19, 12, 20, 18, 10,  5,  5, 15, 12, 17, 12,  6, 13,
         8, 11, 11,  5, 21,  8, 24, 11,  8, 10,  7,  5, 15, 10, 19,  5, 16, 11,
        18, 21, 22, 24, 20, 13,  6, 15, 22, 18,  9,  6, 11, 17, 23, 20,  6,  9,
         9, 15, 23, 23, 14, 19, 19, 15, 17, 18,  6, 20,  6, 11,  6, 10, 10, 11,
         8, 10, 19, 20,  8, 16,  5,  8, 11, 21,  8, 19,  8, 20, 10, 13,  8, 19,
        18,  9,  6, 12, 13, 18,  5, 18, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
        25, 25, 25, 25, 25, 25, 

In [12]:
data_flattened = [item.numpy().flatten() for item in data]

# Assume labels is your (2,) vector labels
labels = np.array(labels)  # convert to numpy array if it's not already

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data_flattened, labels, test_size=0.25, random_state=41)


# Create and train the multi-output SVM model
svm_model = svm.SVC()
multi_output_svm = MultiOutputClassifier(svm_model)
multi_output_svm.fit(X_train, y_train)

# Predict on the test set
y_pred = multi_output_svm.predict(X_test)

# Evaluate the model - you may need to define a suitable evaluation metric for your multi-output task
# For simplicity, here we're using accuracy score which may or may not be suitable for your task
accuracy_list = []
for i in range(no_mask_token):
    accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
    accuracy_list.append(accuracy)
    print(f'Accuracy for output {i + 1}: {accuracy * 100:.2f}%')

print(f'Total Accuracy for output: {np.sum(accuracy_list) * 100/no_mask_token:.2f}%')

Accuracy for output 1: 71.43%
Accuracy for output 2: 60.32%
Accuracy for output 3: 57.14%
Accuracy for output 4: 61.90%
Accuracy for output 5: 66.67%
Total Accuracy for output: 63.49%
